# Apache Iceberg Lab 
## Unit 1: Create a base Parquet table
Create a base table in Parquet, off of the Kaggle Lending Club Loan dataset, preloaded into your GCS data bucket in directory parquet-source.

### 1. Imports

In [1]:
import pandas as pd
from pyspark.sql.functions import col
from pyspark.sql import SparkSession
import warnings

warnings.filterwarnings('ignore')

### 2. Create a Spark session powered by Cloud Dataproc 

In [2]:
spark = SparkSession.builder.appName('Loan Analysis').getOrCreate()
spark

### 3. Declare variables

In [3]:
project_id_output = !gcloud config list --format "value(core.project)" 2>/dev/null
PROJECT_ID = project_id_output[0]
print("PROJECT_ID: ", PROJECT_ID)

PROJECT_ID:  nikhim-iceberg-lab


In [4]:
project_name_output = !gcloud projects describe $PROJECT_ID | grep name | cut -d':' -f2 | xargs
PROJECT_NAME = project_name_output[0]
print("PROJECT_NAME: ", PROJECT_NAME)

PROJECT_NAME:  nikhim-iceberg-lab


In [5]:
project_number_output = !gcloud projects describe $PROJECT_ID | grep projectNumber | cut -d':' -f2 | xargs
PROJECT_NUMBER = project_number_output[0]
print("PROJECT_NUMBER: ", PROJECT_NUMBER)

PROJECT_NUMBER:  928505941962


In [9]:
DATA_LAKE_ROOT_PATH= f"gs://iceberg-data-bucket-{PROJECT_NUMBER}"

In [10]:
RAW_SOURCE_FQ_GCS_PATH = f"{DATA_LAKE_ROOT_PATH}/parquet-source/*"

### 4. Explore the raw loans data

In [11]:
!gsutil ls -r $RAW_SOURCE_FQ_GCS_PATH

gs://iceberg-data-bucket-928505941962/parquet-source/
gs://iceberg-data-bucket-928505941962/parquet-source/loans_raw_1.snappy.parquet
gs://iceberg-data-bucket-928505941962/parquet-source/loans_raw_2.snappy.parquet
gs://iceberg-data-bucket-928505941962/parquet-source/loans_raw_3.snappy.parquet
gs://iceberg-data-bucket-928505941962/parquet-source/loans_raw_4.snappy.parquet


In [12]:
rawDF = spark.read.parquet(RAW_SOURCE_FQ_GCS_PATH)

In [13]:
rawDF.printSchema()

root
 |-- id: string (nullable = true)
 |-- member_id: string (nullable = true)
 |-- loan_amnt: float (nullable = true)
 |-- funded_amnt: integer (nullable = true)
 |-- funded_amnt_inv: double (nullable = true)
 |-- term: string (nullable = true)
 |-- int_rate: string (nullable = true)
 |-- installment: double (nullable = true)
 |-- grade: string (nullable = true)
 |-- sub_grade: string (nullable = true)
 |-- emp_title: string (nullable = true)
 |-- emp_length: string (nullable = true)
 |-- home_ownership: string (nullable = true)
 |-- annual_inc: float (nullable = true)
 |-- verification_status: string (nullable = true)
 |-- loan_status: string (nullable = true)
 |-- pymnt_plan: string (nullable = true)
 |-- url: string (nullable = true)
 |-- desc: string (nullable = true)
 |-- purpose: string (nullable = true)
 |-- title: string (nullable = true)
 |-- zip_code: string (nullable = true)
 |-- addr_state: string (nullable = true)
 |-- dti: float (nullable = true)
 |-- delinq_2yrs: float

In [14]:
rawDF=rawDF.na.drop(subset=["addr_state"])
rawDF.createOrReplaceTempView("loans_raw")

23/02/10 14:55:33 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


In [15]:
# Count total loans
spark.sql("select addr_state as state,loan_status, count(*) as loan_count from loans_raw group by addr_state,loan_status").show()

+-----+------------------+----------+
|state|       loan_status|loan_count|
+-----+------------------+----------+
|   CA|        Fully Paid|     24349|
|   CO|           Current|      4541|
|   CO|        Fully Paid|      3900|
|   NC|           Current|      6529|
|   KY| Late (16-30 days)|        12|
|   NC| Late (16-30 days)|        60|
|   PA|Late (31-120 days)|       215|
|   IN|        Fully Paid|      2468|
|   ME|   In Grace Period|        12|
|   WY|        Fully Paid|       353|
|   CO|   In Grace Period|        70|
|   ND|        Fully Paid|       100|
|   DE|Late (31-120 days)|        18|
|   TX|        Fully Paid|     12662|
|   DC|        Fully Paid|       451|
|   MN|       Charged Off|       763|
|   IL|   In Grace Period|       159|
|   OK|   In Grace Period|        29|
|   TX|   In Grace Period|       316|
|   VT|       Charged Off|        47|
+-----+------------------+----------+
only showing top 20 rows



In [16]:
# How many distinct states?
spark.sql("select count(distinct addr_state) from loans_raw").show(truncate=False)

+--------------------------+
|count(DISTINCT addr_state)|
+--------------------------+
|52                        |
+--------------------------+



### 5. Cleanse the raw data

In [17]:
# Distinct states
spark.sql("select distinct addr_state from loans_raw").collect()

[Row(addr_state='AZ'),
 Row(addr_state='SC'),
 Row(addr_state='LA'),
 Row(addr_state='MN'),
 Row(addr_state='NJ'),
 Row(addr_state='DC'),
 Row(addr_state='OR'),
 Row(addr_state='VA'),
 Row(addr_state='RI'),
 Row(addr_state='KY'),
 Row(addr_state='WY'),
 Row(addr_state='NH'),
 Row(addr_state='MI'),
 Row(addr_state='NV'),
 Row(addr_state='WI'),
 Row(addr_state='ID'),
 Row(addr_state='CA'),
 Row(addr_state='CT'),
 Row(addr_state='NE'),
 Row(addr_state='MT'),
 Row(addr_state='NC'),
 Row(addr_state='VT'),
 Row(addr_state='MD'),
 Row(addr_state='DE'),
 Row(addr_state='MO'),
 Row(addr_state='IL'),
 Row(addr_state='ME'),
 Row(addr_state='WA'),
 Row(addr_state='ND'),
 Row(addr_state='MS'),
 Row(addr_state='AL'),
 Row(addr_state='IN'),
 Row(addr_state='OH'),
 Row(addr_state='TN'),
 Row(addr_state='NM'),
 Row(addr_state='IA'),
 Row(addr_state='PA'),
 Row(addr_state='SD'),
 Row(addr_state='NY'),
 Row(addr_state='TX'),
 Row(addr_state='WV'),
 Row(addr_state='GA'),
 Row(addr_state='MA'),
 Row(addr_s

In [18]:
# Remove data with invalid states
cleansedSubsetDF=spark.sql("select * from loans_raw where addr_state not in ('debt_consolidation')")

In [19]:
# Quick counts
count1=cleansedSubsetDF.count()
print(f"Cleansed subset row count={count1}")

count2=cleansedSubsetDF.select("addr_state").distinct().count()
print(f"Cleansed subset distinct state count={count2}")

Cleansed subset row count=444031


Cleansed subset distinct state count=51


### 6. Persist the cleansed data to the data lake, as Parquet & create an external table definition on it

In [20]:
# Persist the cleaned data
cleansedSubsetDF.coalesce(3).write.format("parquet").mode("overwrite").save(f"{DATA_LAKE_ROOT_PATH}/parquet-cleansed")

In [21]:
# Check if we are using the Dataproc Metastore
spark.sparkContext._conf.get("spark.hive.metastore.uris")

'thrift://10.93.64.15:9080'

In [22]:
# Create a database if it does not exist already
spark.sql("SHOW DATABASES;").show(truncate=False)

ivysettings.xml file not found in HIVE_HOME or HIVE_CONF_DIR,/etc/spark/conf/ivysettings.xml will be used
23/02/10 15:01:40 INFO DependencyResolver: ivysettings.xml file not found in HIVE_HOME or HIVE_CONF_DIR,/etc/spark/conf/ivysettings.xml will be used
23/02/10 15:01:41 INFO metastore: Trying to connect to metastore with URI thrift://10.93.64.15:9080
23/02/10 15:01:41 INFO metastore: Opened a connection to metastore, current connections: 1
23/02/10 15:01:41 INFO metastore: Connected to metastore.


+---------+
|namespace|
+---------+
|default  |
+---------+



In [23]:
# Create a database if it does not exist already
spark.sql("CREATE DATABASE IF NOT EXISTS loan_db;").show(truncate=False)

++
||
++
++



In [24]:
spark.sql("show databases").show()

+---------+
|namespace|
+---------+
|  default|
|  loan_db|
+---------+



In [25]:
# Create an external table defintion on the parquet files
spark.sql("DROP TABLE IF EXISTS loan_db.loans_cleansed_parquet;").show(truncate=False)
spark.sql(f"CREATE TABLE loan_db.loans_cleansed_parquet USING parquet LOCATION '{DATA_LAKE_ROOT_PATH}/parquet-cleansed';").show(truncate=False)

23/02/10 15:01:58 INFO metastore: Trying to connect to metastore with URI thrift://10.93.64.15:9080
23/02/10 15:01:58 INFO metastore: Opened a connection to metastore, current connections: 2
23/02/10 15:01:58 INFO metastore: Connected to metastore.


++
||
++
++



23/02/10 15:01:59 INFO SQLStdHiveAccessController: Created SQLStdHiveAccessController for session context : HiveAuthzSessionContext [sessionString=96fbc719-6cd0-4361-89b8-0180ce31ef04, clientType=HIVECLI]
23/02/10 15:01:59 WARN SessionState: METASTORE_FILTER_HOOK will be ignored, since hive.security.authorization.manager is set to instance of HiveAuthorizerFactory.
23/02/10 15:01:59 INFO metastore: Mestastore configuration hive.metastore.filter.hook changed from org.apache.hadoop.hive.metastore.DefaultMetaStoreFilterHookImpl to org.apache.hadoop.hive.ql.security.authorization.plugin.AuthorizationMetaStoreFilterHook
23/02/10 15:01:59 INFO metastore: Closed a connection to metastore, current connections: 1
23/02/10 15:01:59 INFO metastore: Trying to connect to metastore with URI thrift://10.93.64.15:9080
23/02/10 15:01:59 INFO metastore: Opened a connection to metastore, current connections: 2
23/02/10 15:01:59 INFO metastore: Connected to metastore.
23/02/10 15:01:59 INFO metastore: Try

++
||
++
++



In [26]:
# Review what's in the data lake
!gsutil ls -r $DATA_LAKE_ROOT_PATH

gs://iceberg-data-bucket-928505941962/parquet-cleansed/:
gs://iceberg-data-bucket-928505941962/parquet-cleansed/
gs://iceberg-data-bucket-928505941962/parquet-cleansed/_SUCCESS
gs://iceberg-data-bucket-928505941962/parquet-cleansed/part-00000-90b80b5a-8355-4441-b654-b72820816fea-c000.snappy.parquet
gs://iceberg-data-bucket-928505941962/parquet-cleansed/part-00001-90b80b5a-8355-4441-b654-b72820816fea-c000.snappy.parquet
gs://iceberg-data-bucket-928505941962/parquet-cleansed/part-00002-90b80b5a-8355-4441-b654-b72820816fea-c000.snappy.parquet

gs://iceberg-data-bucket-928505941962/parquet-source/:
gs://iceberg-data-bucket-928505941962/parquet-source/
gs://iceberg-data-bucket-928505941962/parquet-source/loans_raw_1.snappy.parquet
gs://iceberg-data-bucket-928505941962/parquet-source/loans_raw_2.snappy.parquet
gs://iceberg-data-bucket-928505941962/parquet-source/loans_raw_3.snappy.parquet
gs://iceberg-data-bucket-928505941962/parquet-source/loans_raw_4.snappy.parquet


### 7. Create a parquet table on the base parquet dataset

In [27]:
# Remove any residual files from potential prior run
!gsutil rm -rf $DATA_LAKE_ROOT_PATH/parquet-consumable

CommandException: 1 files/objects could not be removed.


In [28]:
# Create table in Parquet off of the cleansed raw data
spark.sql("DROP TABLE IF EXISTS loan_db.loans_by_state_parquet;").show(truncate=False)
spark.sql(f"CREATE TABLE loan_db.loans_by_state_parquet USING parquet LOCATION '{DATA_LAKE_ROOT_PATH}/parquet-consumable' AS SELECT addr_state, count(loan_status) as loan_count FROM loan_db.loans_cleansed_parquet GROUP BY addr_state;")

++
||
++
++



DataFrame[]

In [29]:
# Check the Dataproc metastore for the new table
spark.sql("show tables from loan_db;").show(truncate=False)

+---------+----------------------+-----------+
|namespace|tableName             |isTemporary|
+---------+----------------------+-----------+
|loan_db  |loans_by_state_parquet|false      |
|loan_db  |loans_cleansed_parquet|false      |
|         |loans_raw             |true       |
+---------+----------------------+-----------+



In [30]:
# List some data
spark.sql("select * from loan_db.loans_by_state_parquet").show(truncate=False)

+----------+----------+
|addr_state|loan_count|
+----------+----------+
|AZ        |10318     |
|SC        |5460      |
|LA        |5284      |
|MN        |8031      |
|NJ        |16367     |
|DC        |1059      |
|OR        |5258      |
|VA        |12775     |
|RI        |1968      |
|KY        |4287      |
|WY        |964       |
|NH        |2148      |
|MI        |11638     |
|NV        |6309      |
|WI        |5798      |
|ID        |522       |
|CA        |62090     |
|CT        |6767      |
|NE        |1299      |
|MT        |1220      |
+----------+----------+
only showing top 20 rows



### 8. Review what is in the data lake

Review cell #8. There was just one directory - parquet-source. 

Next review cell #19. A directory called parquet-cleansed was added. 

At the end of this notebook, we also have a parquet-consumable directory

In [31]:
!gsutil ls -r $DATA_LAKE_ROOT_PATH

gs://iceberg-data-bucket-928505941962/parquet-cleansed/:
gs://iceberg-data-bucket-928505941962/parquet-cleansed/
gs://iceberg-data-bucket-928505941962/parquet-cleansed/_SUCCESS
gs://iceberg-data-bucket-928505941962/parquet-cleansed/part-00000-90b80b5a-8355-4441-b654-b72820816fea-c000.snappy.parquet
gs://iceberg-data-bucket-928505941962/parquet-cleansed/part-00001-90b80b5a-8355-4441-b654-b72820816fea-c000.snappy.parquet
gs://iceberg-data-bucket-928505941962/parquet-cleansed/part-00002-90b80b5a-8355-4441-b654-b72820816fea-c000.snappy.parquet

gs://iceberg-data-bucket-928505941962/parquet-consumable/:
gs://iceberg-data-bucket-928505941962/parquet-consumable/
gs://iceberg-data-bucket-928505941962/parquet-consumable/_SUCCESS
gs://iceberg-data-bucket-928505941962/parquet-consumable/part-00000-4d27f2ac-f9cc-4c69-820b-666c9050e00b-c000.snappy.parquet

gs://iceberg-data-bucket-928505941962/parquet-source/:
gs://iceberg-data-bucket-928505941962/parquet-source/
gs://iceberg-data-bucket-9285059419

23/02/10 15:04:43 WARN JavaUtils: Attempt to delete using native Unix OS command failed for path = /var/tmp/spark/local-dir/blockmgr-fd92277c-63e0-4a2b-bfe3-c1863c34a324. Falling back to Java IO way
java.io.IOException: Failed to delete: /var/tmp/spark/local-dir/blockmgr-fd92277c-63e0-4a2b-bfe3-c1863c34a324
	at org.apache.spark.network.util.JavaUtils.deleteRecursivelyUsingUnixNative(JavaUtils.java:171)
	at org.apache.spark.network.util.JavaUtils.deleteRecursively(JavaUtils.java:110)
	at org.apache.spark.network.util.JavaUtils.deleteRecursively(JavaUtils.java:91)
	at org.apache.spark.util.Utils$.deleteRecursively(Utils.scala:1193)
	at org.apache.spark.storage.DiskBlockManager.$anonfun$doStop$1(DiskBlockManager.scala:318)
	at org.apache.spark.storage.DiskBlockManager.$anonfun$doStop$1$adapted(DiskBlockManager.scala:314)
	at scala.collection.IndexedSeqOptimized.foreach(IndexedSeqOptimized.scala:36)
	at scala.collection.IndexedSeqOptimized.foreach$(IndexedSeqOptimized.scala:33)
	at scala.c

We will use the data under the parquet-consumable directory in the next unit, and create Apache Iceberg table off of it.

### THIS CONCLUDES THIS UNIT. PROCEED TO THE NEXT NOTEBOOK